<a href="https://colab.research.google.com/github/akuliber/akuliber.github.io/blob/master/%E8%87%AA%E4%BD%9C%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日経２２５スクリーニング


In [ ]:
import yfinance as yf
import pandas as pd
import datetime
import requests
import time

# 検索条件
MAX_PER = 20  # 20倍
MAX_PBR = 2.0  # 2.0以下
MIN_UP_CHANGE_RATE = 0.10  # 10%以上の上昇
MIN_DOWN_CHANGE_RATE = -0.05  # 5%以上の下落
MIN_DIVIDEND_YIELD = 0.10  # 4%以上の配当利回り
DAYS = 30 #過去何日から起算しての変化率か？

import pandas as pd

def get_nikkei225_tickers():
    # CSVファイルのURL
    url = "https://raw.githubusercontent.com/nakachan-ing/finance-streamlit/master/nikkei.csv"

    # CSVファイルをpandasで読み込む (shift_jisエンコーディングを指定)
    try:
        df = pd.read_csv(url, encoding='shift_jis')
    except Exception as e:
        print(f"CSVファイルの読み込みエラー: {e}")
        return []

    # データフレームの内容を確認
    print(df.head())  # 最初の5行を表示

    # 銘柄コードを抽出し、'.T'を追加
    tickers = df['コード'].astype(str).apply(lambda x: x + '.T').tolist()

    return tickers

# 銘柄コードを取得
tickers = get_nikkei225_tickers()

print(f"取得した銘柄数: {len(tickers)}")



# 上昇・下落銘柄を格納するリスト
up_results = []
down_results = []

# スクリーニング処理
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        per = info.get("trailingPE", None)
        pbr = info.get("priceToBook", None)
        dividend_yield = info.get("dividendYield", None)

        if per is None or pbr is None or dividend_yield is None:
            continue

        # 配当利回りとPER、PBRの条件を満たしている銘柄
        if per <= MAX_PER and pbr <= MAX_PBR and dividend_yield >= MIN_DIVIDEND_YIELD:
            # 過去2週間の株価取得
            end = datetime.datetime.today()
            start = end - datetime.timedelta(days=DAYS)
            hist = stock.history(start=start, end=end)

            if len(hist) < 2:
                continue

            start_price = hist['Close'].iloc[0]
            end_price = hist['Close'].iloc[-1]
            change_rate = (end_price - start_price) / start_price

            # 上昇銘柄
            if change_rate >= MIN_UP_CHANGE_RATE:
                up_results.append({
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{dividend_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",  # Rename the key to "株価変化率"
                    "上昇/下降": "上昇"
                })
            # 下落銘柄
            elif change_rate <= MIN_DOWN_CHANGE_RATE:
                down_results.append({
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{dividend_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",  # Rename the key to "株価変化率"
                    "上昇/下降": "下降"
                })

        # time.sleep(1.5)  # アクセス制限回避のためにスリープ
    except Exception as e:
        print(f"エラー発生: {ticker}, {e}")

#過去何日前から起算して
print(f"過去{DAYS}日前から起算して")

# 上昇銘柄のデータフレーム
df_up = pd.DataFrame(up_results)
print(f"\n◎{MIN_UP_CHANGE_RATE * 100:.0f}%以上上昇した銘柄数: {len(df_up)}")
df_up = df_up.sort_values(by="株価変化率", ascending=False).reset_index(drop=True)
display(df_up)

# 下落銘柄のデータフレーム
df_down = pd.DataFrame(down_results)
# 株価変化率（文字列％）をfloatに変換してソート
df_down['株価変化率_num'] = df_down['株価変化率'].str.replace('%', '').astype(float)
df_down = df_down.sort_values(by="株価変化率_num").reset_index(drop=True)
df_down = df_down.drop(columns=["株価変化率_num"])

print(f"\n◎{MIN_DOWN_CHANGE_RATE * 100:.0f}%以上下落した銘柄数: {len(df_down)}")
display(df_down)





    コード    銘柄名          社名   業種
0  4151  協和キリン    協和キリン(株)  医薬品
1  4502     武田   武田薬品工業(株)  医薬品
2  4503  アステラス  アステラス製薬(株)  医薬品
3  4506  大日本住友  大日本住友製薬(株)  医薬品
4  4507    塩野義    塩野義製薬(株)  医薬品
取得した銘柄数: 225
過去30日前から起算して

◎10%以上上昇した銘柄数: 6


,銘柄コード,PER,PBR,配当利回り,株価変化率,上昇/下降
0,1803.T,13.849787,1.229313,2.49%,15.22%,上昇
1,1801.T,13.369059,1.522885,3.82%,14.75%,上昇
2,3863.T,4.192841,0.278485,1.77%,11.90%,上昇
3,9009.T,6.613803,1.449215,0.80%,11.73%,上昇
4,7003.T,3.429365,1.156826,1.37%,11.07%,上昇
5,1812.T,14.224991,1.346475,3.49%,10.89%,上昇



◎-5%以上下落した銘柄数: 48


,銘柄コード,PER,PBR,配当利回り,株価変化率,上昇/下降
0,6506.T,13.495008,1.772229,2.31%,-21.01%,下降
1,8308.T,11.764707,0.908438,2.12%,-15.62%,下降
2,6724.T,11.534279,0.781540,3.67%,-15.55%,下降
3,3099.T,10.416667,1.114271,2.64%,-15.05%,下降
4,5019.T,10.183017,0.636526,4.02%,-14.93%,下降
5,4004.T,6.337145,0.700748,5.04%,-13.12%,下降
6,8604.T,7.132949,0.674476,5.94%,-12.81%,下降
7,8331.T,13.071582,0.747251,3.60%,-12.54%,下降
8,4578.T,10.706675,1.332721,1.77%,-12.51%,下降
9,8411.T,10.080645,0.851589,3.66%,-12.37%,下降


# プライム市場スクリーニング


In [ ]:
import yfinance as yf
import pandas as pd
import datetime
import requests
import random
from io import BytesIO

# 検索条件
MAX_PER = 15  # 15倍
MAX_PBR = 2.0  # 2.0以下
MIN_UP_CHANGE_RATE = 0.10  # 10%以上の上昇
MIN_DOWN_CHANGE_RATE = -0.05  # 5%以上の下落
MIN_DIVIDEND_YIELD = 0.04  # 4%以上の配当利回り
DAYS = 31  # 過去何日から起算しての変化率か？
PRIME_RANDOM = 333 # ランダムに絞り込むプライム市場の銘柄数

# カスタムヘッダーを定義
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# === JPXのExcelファイルから東証プライム銘柄を取得 ===
xls_url = 'https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls'

# JPXのプライム市場の銘柄をExcelから取得
response = requests.get(xls_url)
xls_file = BytesIO(response.content)

# Excelファイルを読み込む（1枚目のシートを仮定）
df = pd.read_excel(xls_file, sheet_name=0)

# カラム名整形
df.columns = df.columns.str.strip()
df = df.dropna(subset=['コード'])  # コードがNaNの行を除外

# プライム市場に絞り込み
prime_df = df[df['市場・商品区分'].str.contains('プライム', na=False)]

# 銘柄コードを yfinance 形式に（4桁 + ".T"）
tickers = prime_df['コード'].astype(str).str.zfill(4) + '.T'

# ランダムに333銘柄を抽出
tickers_list = random.sample(tickers.tolist(), PRIME_RANDOM)

# 取得銘柄数
print(f"取得銘柄数: {len(tickers_list)}")

# 上昇・下落銘柄を格納するリスト
up_results = []
down_results = []

# スクリーニング処理
for ticker in tickers_list:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        per = info.get("trailingPE", None)
        pbr = info.get("priceToBook", None)
        dividend_yield = info.get("dividendYield", None)

        if per is None or pbr is None or dividend_yield is None:
            continue

        # 配当利回りとPER、PBRの条件を満たしている銘柄
        if per <= MAX_PER and pbr <= MAX_PBR and dividend_yield >= MIN_DIVIDEND_YIELD:
            # 過去2週間の株価取得
            end = datetime.datetime.today()
            start = end - datetime.timedelta(days=DAYS)
            hist = stock.history(start=start, end=end)

            if len(hist) < 2:
                continue

            start_price = hist['Close'].iloc[0]
            end_price = hist['Close'].iloc[-1]
            change_rate = (end_price - start_price) / start_price

            # 上昇銘柄
            if change_rate >= MIN_UP_CHANGE_RATE:
                up_results.append({
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{dividend_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",  # Rename the key to "株価変化率"
                    "上昇/下降": "上昇"
                })
            # 下落銘柄
            elif change_rate <= MIN_DOWN_CHANGE_RATE:
                down_results.append({
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{dividend_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",  # Rename the key to "株価変化率"
                    "上昇/下降": "下降"
                })

        # time.sleep(1.5)  # アクセス制限回避のためにスリープ
    except Exception as e:
        print(f"エラー発生: {ticker}, {e}")

# 過去何日前から起算して
print(f"過去{DAYS}日前から起算して")

# 上昇銘柄のデータフレーム
df_up = pd.DataFrame(up_results)
print(f"\n◎{MIN_UP_CHANGE_RATE * 100:.0f}%以上上昇した銘柄数: {len(df_up)}")
df_up = df_up.sort_values(by="株価変化率", ascending=False).reset_index(drop=True)
display(df_up)


# 下落銘柄のデータフレーム
df_down = pd.DataFrame(down_results)

# 株価変化率（文字列％）をfloatに変換してソート
df_down['株価変化率_num'] = df_down['株価変化率'].str.replace('%', '').astype(float)
df_down = df_down.sort_values(by="株価変化率_num").reset_index(drop=True)
df_down = df_down.drop(columns=["株価変化率_num"])

print(f"\n◎{MIN_DOWN_CHANGE_RATE * 100:.0f}%以上下落した銘柄数: {len(df_down)}")
display(df_down)


取得銘柄数: 333
過去31日前から起算して

◎10%以上上昇した銘柄数: 6


,銘柄コード,PER,PBR,配当利回り,株価変化率,上昇/下降
0,2692.T,13.755178,0.987236,1.75%,21.41%,上昇
1,1803.T,13.849787,1.229313,2.49%,15.22%,上昇
2,9267.T,14.992749,1.902268,0.41%,12.65%,上昇
3,7003.T,3.429365,1.156826,1.37%,11.07%,上昇
4,1815.T,13.623844,0.555921,4.40%,11.02%,上昇
5,1812.T,14.224991,1.346475,3.49%,10.89%,上昇



◎-5%以上下落した銘柄数: 44


,銘柄コード,PER,PBR,配当利回り,株価変化率,上昇/下降
0,6724.T,11.534279,0.781540,3.67%,-15.55%,下降
1,8242.T,5.279886,0.796029,2.08%,-14.77%,下降
2,7354.T,13.024036,0.785069,4.51%,-14.02%,下降
3,8604.T,7.132949,0.674476,5.94%,-12.81%,下降
4,8708.T,12.936714,0.903810,2.01%,-12.61%,下降
5,4578.T,10.706675,1.332721,1.77%,-12.51%,下降
6,8411.T,10.080645,0.851589,3.66%,-12.37%,下降
7,6914.T,9.643262,1.088492,2.94%,-11.87%,下降
8,8570.T,12.936753,0.543159,4.53%,-11.33%,下降
9,5857.T,10.351931,1.066998,4.55%,-11.21%,下降
